In [5]:
from architecture.deep_binary_classifier import DeepBinaryClassifier
from architecture.ripper_node           import make_ripper_node
from architecture.lut_node              import make_lut_node

from sklearn.model_selection                import train_test_split
from sklearn.metrics                        import accuracy_score

import pandas as pd
import numpy as np


df   = pd.read_csv("./data/100_bit_artificial/1a.csv")
X,y  = df.drop(columns="class").to_numpy(bool), df["class"].to_numpy(bool)
Xtr,Xte,Ytr,Yte = train_test_split(X,y,test_size=0.2,random_state=1,stratify=y)

In [6]:
config = dict(nodes_per_layer=[32]*5, bits_per_node=[6]*6, rng=42, reuse_prev_width=True)

In [3]:
# the LUT network runs faster on a single thread

lut_net = DeepBinaryClassifier(**config, node_factory=make_lut_node, n_jobs=1)
%time lut_net.fit(Xtr, Ytr)
acc_lut = accuracy_score(Yte, lut_net.predict(Xte))
print(f"LUT network  accuracy: {acc_lut:.4f}")

CPU times: user 88.8 ms, sys: 46.9 ms, total: 136 ms
Wall time: 339 ms
LUT network  accuracy: 0.7275


In [7]:
# the Ripper nodes profit from parallelization

rip_net = DeepBinaryClassifier(**config, node_factory=make_ripper_node, n_jobs=16, extra_node_kw={"ripper_kwargs": {"k": 2}})
%time rip_net.fit(Xtr, Ytr)
acc_rip = accuracy_score(Yte, rip_net.predict(Xte))
print(f"Rule network accuracy: {acc_rip:.4f}")

CPU times: user 458 ms, sys: 131 ms, total: 589 ms
Wall time: 30.1 s
Rule network accuracy: 0.8200
